In [ ]:
import tensorflow as tf
import numpy as np
import os
import cv2
import numpy as np
import random as rn
import tensorflow as tf
import threading
import time

global n_classes
n_classes = 50


In [ ]:
#==========================================================================
#=============Reading data in multithreading manner========================
#==========================================================================
def read_labeled_image_list(image_list_file):
    """Reads a .txt file containing pathes and labeles
    Args:
       image_list_file: a .txt file with one /path/to/image per line
       label: optionally, if set label will be pasted after each line
    Returns:
       List with all filenames in file image_list_file
    """
    f = open(image_list_file, 'r')
    filenames = []
    labels = []

    for line in f:
        filename, label = line[:-1].split(' ')
        filenames.append(filename)
        labels.append(int(label))
        
    return filenames, labels
    
    
def read_images_from_disk(input_queue, size1=512):
    """Consumes a single filename and label as a ' '-delimited string.
    Args:
      filename_and_label_tensor: A scalar string tensor.
    Returns:
      Two tensors: the decoded image, and the string label.
    """
    label = input_queue[1]
    fn=input_queue[0]
    file_contents = tf.read_file(input_queue[0])
    example = tf.image.decode_jpeg(file_contents, channels=3)
    example=tf.image.resize_images(example, [size1,size1])
    return example, label, fn


def setup_inputs(sess, filenames, image_size=256, crop_size=224, isTest=False, batch_size=64):
    
    # Read each image file
    image_list, label_list = read_labeled_image_list(filenames)

    images = tf.cast(image_list, tf.string)
    labels = tf.cast(label_list, tf.int64)
     # Makes an input queue
    if isTest is False:
        isShuffle = True
        numThr = 4
    else:
        isShuffle = False
        numThr = 1
        
    input_queue = tf.train.slice_input_producer([images, labels], shuffle=isShuffle)
    image, y,fn = read_images_from_disk(input_queue)

    channels = 3
    image.set_shape([None, None, channels])
        
    if isTest is False:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_flip_up_down(image)
        image = tf.image.random_saturation(image, .95, 1.05)
        image = tf.image.random_brightness(image, .05)
        image = tf.image.random_contrast(image, .95, 1.05)
        image = tf.random_crop(image, [crop_size, crop_size, 3])
    else:
        image = tf.random_crop(image, [crop_size, crop_size, 3])
    
    image = tf.cast(image, tf.float32)/255.0
    
    image, y,fn = tf.train.batch([image, y, fn], batch_size=batch_size, capacity=batch_size*3, num_threads=numThr, name='labels_and_images')
    tf.train.start_queue_runners(sess=sess)

    return image, y, len(label_list)


In [ ]:
def activation(x,name="activation"):
    return tf.nn.relu(x, name=name)
    
def conv2d(name, l_input, channels, ks, s, p, tst):
    pre_channels=l_input.get_shape().as_list()[3]
    w,b =initializer(pre_channels, channels, name+"conv", ksize=1)
    
    l_input = tf.nn.conv2d(l_input, w, strides=[1,s,s,1], padding=p, name=name)
    l_input = activation(l_input+b)
    l_input = batchnorm(l_input, tst, name)
    return l_input

def batchnorm(conv, isTraining, name='bn'):
    return tf.layers.batch_normalization(conv, training=isTraining, momentum = 0.9, name="bn"+name)

def initializer(in_filters, out_filters, name, ksize=3):
    w1 = tf.get_variable(name+"W", [ksize, ksize, in_filters, out_filters], initializer=tf.glorot_uniform_initializer())
    b1 = tf.get_variable(name+"B", [out_filters], initializer=tf.glorot_uniform_initializer())
    return w1, b1

def max_pool(name, l_input, k, s):
    return tf.layers.max_pooling2d(l_input, k,s, padding='VALID', name=name+'p')

In [ ]:
   
def vgg16(_X, batch_size, tst):
    conv1 = conv2d('conv1', _X, 64, 3, 1, 'SAME', tst)
    conv2 = conv2d('conv2', conv1, 64, 3, 1, 'SAME', tst)
    pool3 = max_pool('pool3', conv2, k=3,s=2) 
    conv3 = conv2d('conv3', pool3, 128, 3, 1, 'SAME', tst)
    conv4 = conv2d('conv4', conv3, 128, 3, 1, 'SAME', tst)
    pool5 = max_pool('pool5', conv4, k=3,s=2)
    conv6 = conv2d('conv6', pool5, 256, 3, 1, 'SAME', tst)
    conv7 = conv2d('conv7', conv6, 256, 3, 1, 'SAME', tst)
    pool8 = max_pool('pool8', conv7, k=3,s=2)
    conv9 = conv2d('conv9', pool8, 512, 3, 1, 'SAME', tst)
    conv10 = conv2d('conv10', conv9, 512, 3, 1, 'SAME', tst)
    conv11 = conv2d('conv11', conv10, 512, 3, 1, 'SAME', tst)
    pool12 = max_pool('pool12', conv11, k=3,s=2)
    conv13 = conv2d('conv13', pool12, 512, 3, 1, 'SAME', tst)
    conv14 = conv2d('conv14', conv13, 512, 3, 1, 'SAME', tst)
    conv15 = conv2d('conv15', conv14, 512, 3, 1, 'SAME', tst)
    pool16 = max_pool('pool16', conv15, k=3,s=2)

    # Find current size of conv5 to fit the requirement of FC1.
    sizes = pool16.get_shape().as_list()
    neurons =  sizes[1]*sizes[2]*sizes[3]
    dense1 = tf.reshape(pool16, [batch_size, neurons]) # Reshape conv5 output to fit dense layer input
    wei_den1 = tf.get_variable("FC14w", [neurons, 4096], initializer=tf.glorot_uniform_initializer())
    b_den1 = tf.get_variable("FC14b", [4096], initializer=tf.glorot_uniform_initializer())
    dense1 = tf.nn.relu(tf.matmul(dense1, wei_den1) + b_den1, name='fc1') # Relu activation
    
    wei_den2 = tf.get_variable("FC15w", [4096, 4096], initializer=tf.glorot_uniform_initializer())
    b_den2 = tf.get_variable("FC15b", [4096], initializer=tf.glorot_uniform_initializer())
    dense2 = tf.nn.relu(tf.matmul(dense1, wei_den2) + b_den2, name='fc2') # Relu activation
    
    wei_den3 = tf.get_variable("FC16w", [4096, n_classes], initializer=tf.glorot_uniform_initializer())
    b_den3 = tf.get_variable("FC16b", [n_classes], initializer=tf.glorot_uniform_initializer())
    out = tf.matmul(dense2, wei_den3) + b_den3 

    return out

In [ ]:
# Training setting
batch_size = 4
display_step = 80

In [ ]:
learning_rate = tf.placeholder(tf.float32)      # Learning rate to be fed
lr = 1e-2
sess = tf.Session()


# Learning rate start

train_data, train_lab, glen1 = setup_inputs(sess,'train.txt', batch_size=batch_size)
val_data, val_lab, vlen1 =  setup_inputs(sess,'val.txt', isTest=True, batch_size=batch_size)
training_iters = glen1*600

with tf.variable_scope("Vgg16Net") as scope:
        pred = vgg16(train_data,batch_size, True)
        scope.reuse_variables()
        valpred = vgg16(val_data,batch_size, False)
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    cost = tf.losses.sparse_softmax_cross_entropy(labels=train_lab, logits=pred)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
correct_prediction = tf.equal(tf.argmax(pred, 1), train_lab)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

correct_prediction2 = tf.equal(tf.argmax(valpred, 1), val_lab)
accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))

In [ ]:

sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()

step = 0
while step * batch_size < training_iters:
    epoch1=np.floor((step*batch_size)/glen1)

    if (((step*batch_size)%glen1 < batch_size) & (lr==1e-3) & (epoch1 >500)):
        lr /= 10

    # Get a batch
    sess.run(optimizer,  feed_dict={learning_rate: lr})

    if (step % 15000==1) & (step>15000):
        save_path = saver.save(sess, "./tf_vgg16_model_iter" + str(step) + ".ckpt")
        print("Model saved in file at iteration %d: %s" % (step*batch_size,save_path))

    if step % display_step == 1:
        # calculate the loss
        loss,acc = sess.run([cost, accuracy])
        acc2 = []
        
        for j in range(int(np. ceil(vlen1 / batch_size))):
            acc2.append(sess.run(accuracy2))
        acc2 = np.mean(acc2)

        print("Iter=%d/epoch=%d, Training Accuracy=%.6f, Loss=%.6g,Test Accuracy=%.6f, lr=%f" % (step*batch_size, epoch1, acc, loss, acc2, lr))
    step += 1
print("Optimization Finished!")
save_path = saver.save(sess, "./tf_vgg16_model.ckpt")
print("Model saved in file: %s" % save_path)

In [ ]:
exit()